In [38]:
import numpy as np 
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pickle
import random
from copy import copy, deepcopy
from scipy.io import arff
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.spatial.distance import pdist, cdist
from sklearn.metrics.pairwise import euclidean_distances, pairwise_distances
from sklearn.utils import check_random_state
from scipy.spatial import distance
from tslearn.soft_dtw_fast import *
from tslearn.cysax import *
from tslearn.metrics import dtw, dtw_path,dtw_path_from_metric,soft_dtw
from tslearn.utils import *


# Loading Training and Testing Data Source

In [39]:
data1 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension1_TRAIN.arff')
data2 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension2_TRAIN.arff')
data3 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension3_TRAIN.arff')

Test_data1 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension1_TEST.arff')
Test_data2 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension2_TEST.arff')
Test_data3 = arff.loadarff('/Users/arslanaliawan/Desktop/Pre-experiment/Multivariate_arff/CharacterTrajectories/CharacterTrajectoriesDimension3_TEST.arff')

In [40]:
df_D1 = pd.DataFrame(data1[0])
df_D2 = pd.DataFrame(data2[0])
df_D3 = pd.DataFrame(data3[0])

Test_df_D1 = pd.DataFrame(Test_data1[0])
Test_df_D2 = pd.DataFrame(Test_data2[0])
Test_df_D3 = pd.DataFrame(Test_data3[0])

## Make Over the data, as desired

In [41]:
df_D1 =df_D1.apply(lambda x : x.fillna(method='ffill'), axis=1) 
df_D2 =df_D2.apply(lambda x : x.fillna(method='ffill'), axis=1)
df_D3 =df_D3.apply(lambda x : x.fillna(method='ffill'), axis=1) 

Test_df_D1 =Test_df_D1.apply(lambda x : x.fillna(method='ffill'), axis=1) 
Test_df_D2 =Test_df_D2.apply(lambda x : x.fillna(method='ffill'), axis=1)
Test_df_D3 =Test_df_D3.apply(lambda x : x.fillna(method='ffill'), axis=1) 

In [42]:
df_D1 = df_D1.drop('target', axis=1)
df_D2 = df_D2.drop('target', axis=1)
df_D3 = df_D3.drop('target', axis=1)

Test_df_D1 = Test_df_D1.drop('target', axis=1)
Test_df_D2 = Test_df_D2.drop('target', axis=1)
Test_df_D3 = Test_df_D3.drop('target', axis=1)

In [43]:
Sort_Dim_1 = np.expand_dims(df_D1.values.reshape(-1),axis=1)
Sort_Dim_2 = np.expand_dims(df_D2.values.reshape(-1),axis=1)
Sort_Dim_3 = np.expand_dims(df_D3.values.reshape(-1),axis=1)

Sort_Test_Dim_1 = np.expand_dims(Test_df_D1.values.reshape(-1),axis=1)
Sort_Test_Dim_2 = np.expand_dims(Test_df_D2.values.reshape(-1),axis=1)
Sort_Test_Dim_3 = np.expand_dims(Test_df_D3.values.reshape(-1),axis=1)

In [44]:
Training_View = np.concatenate((Sort_Dim_1, Sort_Dim_2),axis=1)
Training_View = np.concatenate((Training_View, Sort_Dim_3),axis=1)

In [45]:
Training_View.shape

(258804, 3)

In [46]:
Testing_View = np.concatenate((Sort_Test_Dim_1, Sort_Test_Dim_2),axis=1)
Testing_View = np.concatenate((Testing_View, Sort_Test_Dim_3),axis=1)

In [126]:
finalarray_test_data = Testing_View[:][0:182]

In [127]:
finalarray_test_data.shape

(182, 3)

## Initializing the data

In [149]:
Total_rows = 261352
Sample_Percentage = 0
Number_of_run= 0
Length_RTS = 0
global Sub_Sample_copy_PTS 
Training_Results_List = []
PTS_Results_List = []
Store_Avg_list = []
dictionary = dict()
dictionary_RTS = dict()
dictionary_RTS_all = dict()
dictionary_PTS = dict()
dictionary_PTS_for_all_change = dict()
RTS_short_list_Results_List = []
Starting_point = Testing_View[:][0:1]
End_point = Testing_View[:][181:182]

## Functions body for Random TS

In [150]:
def Calculate_Percentage(Sample_Percentage):
    Refining_Percentage= Total_rows * (Sample_Percentage/100)
    Refining_Percentage = round(Refining_Percentage)   
    return Refining_Percentage

In [151]:
def Random_Part(Refining_Percentage):
    Sub_Sampling = Training_View[:][0:Refining_Percentage]
    return Sub_Sampling    

In [152]:
def Get_random_row(Sub_Sampling,Length_RTS):
    
    number_of_rows = Sub_Sampling.shape[0]
    random_indices = np.random.choice(number_of_rows, size=Length_RTS, replace=False)
    random_rows = Sub_Sampling[random_indices, :]
    return random_rows
    

In [153]:
def Adding_start_end_point(random_rows):
    Output_TS = np.concatenate((Starting_point, random_rows),axis=0)
    Output_TS = np.concatenate((Output_TS, End_point),axis=0)
    return Output_TS

In [154]:
def Calculate_Random_DTW(Output_TS):
    path, dist = dtw_path(finalarray_test_data, Output_TS)
    if(dist>0):
        Training_Results_List.append(dist)  


# CODE 

In [155]:
Given_Percentage = 20
random_num_out = 60

In [156]:
for y in range(1,10):
    Refining_Percentage = Calculate_Percentage(Given_Percentage)
    Sub_Sampling = Random_Part(Refining_Percentage)
    Iteration_number = len(Sub_Sampling) /182
    Iteration_number_t = round (Iteration_number)

    for i in range (1,Iteration_number_t):

        random_rows = Get_random_row(Sub_Sampling,random_num_out)
        Output_TS = Adding_start_end_point(random_rows)
        Calculate_Random_DTW(Output_TS) 
        
    dictionary_RTS_all[y] = Training_Results_List
    Training_Results_List= []

In [157]:
dictionary_RTS_all

{1: [12.486529367173167,
  14.299926316513274,
  13.376318166473764,
  14.528277188832543,
  15.914638554128835,
  14.760450788755369,
  12.555718127088392,
  14.50304196255075,
  15.732505429736674,
  14.90031544314247,
  14.471690529974099,
  13.711722598286332,
  13.881463675816287,
  15.959448943959567,
  14.637606032096986,
  15.618795499131071,
  14.519890841680285,
  14.019934045145934,
  14.347229220126405,
  13.820306031851361,
  14.768360125935207,
  14.827723817004081,
  15.231888961748538,
  13.290994520540703,
  16.07974382425827,
  13.383505730903767,
  14.832572344132995,
  13.44459922897574,
  14.917760797314592,
  15.222036817881868,
  14.585089162763285,
  14.05762682229558,
  16.557859643278476,
  14.103785294226725,
  14.389492033608729,
  14.955536489928706,
  15.53222005873114,
  14.334644047249764,
  13.838540287250998,
  14.217550885668352,
  15.004638039910155,
  14.412342848139023,
  13.44577538192636,
  13.39023293052279,
  14.07551402752944,
  14.29304751437

## Functions body for Pseudo TS

In [19]:
def generate_similarity_matrix(Complete_PTS):
    ED_Matrix = distance.cdist(Complete_PTS, Complete_PTS, 'euclidean')
    return ED_Matrix

In [20]:
def generate_graph(similarity_output):
    G = nx.from_numpy_matrix(similarity_output, create_using=nx.Graph)
    layout = nx.spring_layout(G)
    sizes = len(similarity_output)
    return G

In [21]:
def generate_MST(G):
    layout = nx.spring_layout(G)
    T=nx.minimum_spanning_tree(G)
    return T

In [22]:
def generate_Dijkstra(T):
    Dij = nx.dijkstra_path(T,0,181)
    return Dij

In [23]:
def doing_miscellaneous_work(Dijkstra_output):
    a = []
    for x in range(len(Dijkstra_output)):
    
        Index = Dijkstra_output[x]
        a.append(list(Complete_PTS[Index]))
    P_TS = np.array(a)
    return P_TS

In [24]:
def generate_DTW_PTS(Miscellaneous_output):
    path, dist = dtw_path(finalarray_test_data, Miscellaneous_output)
    if(dist>0):
        PTS_Results_List.append(dist)     

In [25]:
def Adding_start_end_point_PTS(Pseudo_TS,y):
    if(y==1):
        SP = [Testing_View[:][0]]
        EP = [Testing_View[:][181]]
        Output_PTS = np.concatenate((SP, Pseudo_TS),axis=0)
        Output_PTS = np.concatenate((Output_PTS, EP),axis=0)
    else:
        p =182
        SP = [Testing_View[:][p*(y-1)]]
        EP = [Testing_View[:][((y*p)-1)]]
        Output_PTS = np.concatenate((SP, Pseudo_TS),axis=0)
        Output_PTS = np.concatenate((Output_PTS, EP),axis=0)
    return Output_PTS

In [26]:
def Run_all_fun(Complete_PTS):
    similarity_output = generate_similarity_matrix(Complete_PTS)
    graph_output = generate_graph(similarity_output)
    MST_output_1 =  generate_MST(graph_output)
    Dijkstra_output = generate_Dijkstra(MST_output_1)
    Miscellaneous_output = doing_miscellaneous_work(Dijkstra_output)
    generate_DTW_PTS(Miscellaneous_output)

### RTS with Number of iterations ; different percentage ; Diff. length of RTS

In [27]:
for i in range (1,11):
#     random_num_out = generate_random_number_for_RTS()
    for x in range(1,6):
        
        Refining_Percentage = Calculate_Percentage(20*x)
        Sub_Sampling = Random_Part(Refining_Percentage)
        random_rows = Get_random_row(Sub_Sampling,random_num_out)
        Output_TS = Adding_start_end_point(random_rows)
        Calculate_Random_DTW(Output_TS,random_num_out) 
    dictionary_RTS_all[i] = RTS_short_list_Results_List
    RTS_short_list_Results_List= []

### Changing whole Training TS with SP and EP for PTS

In [101]:
for y in range(1,11):
    for x in range(1,6):
        i = 181
        if(x==1):
            Refining_Percentage = Calculate_Percentage(20*x)
            Sub_Sampling = Random_Part(Refining_Percentage)
            Sub_Sampling_PTS = Sub_Sampling[:][1:181*x]
            Pseudo_TS = Sub_Sampling_PTS
            Complete_PTS = Adding_start_end_point_PTS(Pseudo_TS,y)
            Run_all_fun(Complete_PTS) 

        else:
            Refining_Percentage = Calculate_Percentage(20*x)
            Sub_Sampling = Random_Part(Refining_Percentage)
            Sub_Sampling_PTS = Sub_Sampling[:][1+i:181*x]
            Pseudo_TS = Sub_Sampling_PTS
            Complete_PTS = Adding_start_end_point_PTS(Pseudo_TS,y)
            Run_all_fun(Complete_PTS) 
            
    dictionary_PTS_for_all_change[y] = PTS_Results_List
    PTS_Results_List = []